In [13]:
import csv
from itertools import batched

from dotenv import load_dotenv
from tqdm.notebook import tqdm

from bigger_picker.rayyan import RayyanManager

load_dotenv()

True

In [14]:
def get_csv_ratings(file_path):
    dois = set()
    with open(file_path) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            dois.add(row["doi"])

    dois = {doi.removeprefix("https://doi.org/").lower().strip() for doi in dois}
    return dois

In [19]:
included_ft_dois = get_csv_ratings("ft_included.csv")
excluded_ft_dois = get_csv_ratings("ft_excluded.csv")
excluded_ab_dois = get_csv_ratings("ab_excluded.csv")
missing_ft_dois = get_csv_ratings("ft_missing.csv")

# Combine the ones that indicate inclusion at the abstract level
included_ab_dois = included_ft_dois.union(excluded_ft_dois).union(missing_ft_dois)
print(f"Included at abstract level: {len(included_ab_dois)}")
print(f"Included at full text level: {len(included_ft_dois)}")

Included at abstract level: 5679
Included at full text level: 2828


In [20]:
rayyan = RayyanManager("../rayyan_tokens.json")

In [21]:
results_params = {"start": 0, "length": 10}
results = rayyan.review.results(rayyan.review_id, results_params)
print(f"Total records in Rayyan review: {results['recordsTotal']}")

Total records in Rayyan review: 31639


In [22]:
n_articles = results["recordsTotal"]
batch_size = 1000
batches = batched(range(0, n_articles), batch_size)
total_batches = (n_articles + batch_size - 1) // batch_size

with tqdm(total=total_batches, desc="Overall Progress") as overall_pbar:
    for batch_idx, batch in enumerate(batches):
        results_params = {"start": batch[0], "length": len(batch)}
        articles = rayyan.review.results(rayyan.review_id, results_params)
        articles_data = articles["data"]

        with tqdm(
            total=len(articles_data),
            desc=f"Batch {batch_idx + 1}/{total_batches}",
            leave=False,
        ) as batch_pbar:
            for article in articles_data:
                article_doi = article["doi"]
                try:
                    article_doi = (
                        article_doi.removeprefix("https://doi.org/").lower().strip()
                    )
                except AttributeError:
                    # No DOI so skip
                    batch_pbar.update(1)
                    continue

                plan = {}

                if article_doi in included_ft_dois:
                    plan["OttoSR: Include Full-Text"] = 1
                    plan["OttoSR: Include Abstract"] = 1
                if article_doi in included_ab_dois:
                    plan["OttoSR: Include Abstract"] = 1
                if article_doi in excluded_ft_dois:
                    plan["OttoSR: Exclude Full-Text"] = 1
                    plan["OttoSR: Include Abstract"] = 1
                if article_doi in excluded_ab_dois:
                    plan["OttoSR: Exclude Abstract"] = 1

                # Check if we can avoid processing
                existing_labels = article["customizations"].get("labels", {})

                if (
                    existing_labels.get("OttoSR: Include")
                    == plan.get("OttoSR: Include")
                    and existing_labels.get("OttoSR: Include Abstract")
                    == plan.get("OttoSR: Include Abstract")
                    and existing_labels.get("OttoSR: Exclude Abstract")
                    == plan.get("OttoSR: Exclude Abstract")
                    and existing_labels.get("OttoSR: Include Full-Text")
                    == plan.get("OttoSR: Include Full-Text")
                    and existing_labels.get("OttoSR: Exclude Full-Text")
                    == plan.get("OttoSR: Exclude Full-Text")
                ):
                    # There is no existing label or plan for this article
                    included_ab_dois.discard(article_doi)
                    included_ft_dois.discard(article_doi)
                    excluded_ft_dois.discard(article_doi)
                    batch_pbar.update(1)
                    continue

                # If we haven't set the plan, we default to removing existing labels
                plan.setdefault("OttoSR: Include", -1)
                plan.setdefault("OttoSR: Include Abstract", -1)
                plan.setdefault("OttoSR: Exclude Abstract", -1)
                plan.setdefault("OttoSR: Include Full-Text", -1)
                plan.setdefault("OttoSR: Exclude Full-Text", -1)

                try:
                    rayyan.review.customize(rayyan.review_id, article["id"], plan)
                    included_ab_dois.discard(article_doi)
                    included_ft_dois.discard(article_doi)
                    excluded_ft_dois.discard(article_doi)
                except Exception as e:
                    print(f"Error updating article {article['id']}: {e}")

                batch_pbar.update(1)

        overall_pbar.update(1)

Overall Progress:   0%|          | 0/32 [00:00<?, ?it/s]

Batch 1/32:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 2/32:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 3/32:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 4/32:   0%|          | 0/1000 [00:00<?, ?it/s]

...

Batch 28/32:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 29/32:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 30/32:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 31/32:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 32/32:   0%|          | 0/639 [00:00<?, ?it/s]

In [23]:
print("Included abstract DOIs left:", len(included_ab_dois))
print("Included full-text DOIs left:", len(included_ft_dois))
print("Excluded DOIs left:", len(excluded_ft_dois))


Included abstract DOIs left: 101
Included full-text DOIs left: 36
Excluded DOIs left: 34


In [ ]:
def get_leftover_dois(csv_paths: list[str], leftover_dois: set):
    leftover_rows = []

    for csv_path in csv_paths:
        with open(csv_path) as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if (
                    row["doi"].removeprefix("https://doi.org/").lower().strip()
                    in leftover_dois
                ):
                    leftover_rows.append(row)

    return leftover_rows


leftover_ab_dois = get_leftover_dois(
    [
        "Sanders-included.csv",
        "Sanders - excluded-ft.csv",
        "Sanders - failed-ft.csv",
        "Sanders - missing-ft.csv",
    ],
    included_ab_dois,
)
assert len(leftover_ab_dois) == len(included_ab_dois)

leftover_ft_dois = get_leftover_dois(
    ["Sanders-included.csv"],
    included_ft_dois,
)
assert len(leftover_ft_dois) == len(included_ft_dois)

leftover_excludedft_dois = get_leftover_dois(
    ["Sanders - excluded-ft.csv"],
    excluded_ft_dois,
)
assert len(leftover_excludedft_dois) == len(excluded_ft_dois)


In [ ]:
for article in leftover_ab_dois:
    print(
        f"""
Article DOI: {article["doi"]}
Title: {article["title"]}
Authors: {article["authors"]}
Year: {article["date"]}
"""
    )

In [ ]:
for article in leftover_ft_dois:
    print(
        f"""
Article DOI: {article["doi"]}
Title: {article["title"]}
Authors: {article["authors"]}
Year: {article["date"]}
"""
    )

In [ ]:
for article in leftover_excludedft_dois:
    print(
        f"""
Article DOI: {article["doi"]}
Title: {article["title"]}
Authors: {article["authors"]}
Year: {article["date"]}
"""
    )